# Import Essential Libraries 

In [ ]:
pip install keras-tuner --upgrade

In [48]:
# Import Liraries
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
from pandas import read_csv
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

# Dataset Loading 

In [49]:
# Load Data 
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data'
df = read_csv(url, header=None)
dataset = df.values

In [50]:
# Defining Column names 
df.columns = ["Sex", "Length", "Diameter", "Height", "Whole.weight", "Shucked.weight", "Viscera.weight", "Shell.weight", "Rings"]

# Data Preprocessing 

In [51]:
# Reference: https://www.geeksforgeeks.org/ml-one-hot-encoding-of-datasets-in-python/#:~:text=One%20Hot%20Encoding%20is%20used,takes%20only%20numerical%20categorical%20values. 
# Input Column Encoding: Encoding the Sex column by One Hot Encoding using the get_dummies() function of the pandas 
df_dummies = pd.get_dummies(df['Sex'], drop_first = True, prefix="Sex_")

# Inserting dummy columns
for column in df_dummies.columns:
    df[column] = df_dummies[column]
    
# Dropping the original column
df = df.drop(columns = ['Sex'])

In [52]:
# Output Column Encoding: Rings Column Encoding

def rings_label(x):
    if x<=10:
        return 'Young'
    if x<=20:
        return 'Middle-Aged'
    if x<=30:
        return 'Old'
    
df['Rings'] = df['Rings'].apply(rings_label)

In [53]:
df['Rings']

0       Middle-Aged
1             Young
2             Young
3             Young
4             Young
           ...     
4172    Middle-Aged
4173          Young
4174          Young
4175          Young
4176    Middle-Aged
Name: Rings, Length: 4177, dtype: object

In [54]:
# Applying One-Hot Encoding to the Ring Column again to handle the imbalanced data
df_dummies = pd.get_dummies(df['Rings'])

# Inserting dummy columns
for column in df_dummies.columns:
    df[column] = df_dummies[column]
    
# Dropping the original column
df = df.drop(columns = ['Rings'])

# Train-Validation-Test Split

In [55]:
# defining the input and output columns to separate the dataset 

input_columns = df.columns.tolist()
input_columns.remove('Young')
input_columns.remove('Middle-Aged')
input_columns.remove('Old')

output_columns = ['Young', 'Middle-Aged', 'Old']

In [56]:
# Reference: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html 
# Splitting into train, val and test set
train_df, val_test_df = train_test_split(df, test_size = 0.5)
val_df, test_df = train_test_split(val_test_df, test_size = 0.5)

# Splitting into X as input and y as output
Xtrain, ytrain = np.array(train_df[input_columns]), np.array(train_df[output_columns])
Xval, yval = np.array(val_df[input_columns]), np.array(val_df[output_columns])
Xtest, ytest = np.array(test_df[input_columns]), np.array(test_df[output_columns])

# Standardization

In [57]:
# Reference: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html 
ss = StandardScaler()
Xtrain = ss.fit_transform(Xtrain)
Xval = ss.transform(Xval)
Xtest = ss.transform(Xtest)

# DNN Model

In [58]:
# Reference: https://keras.io/guides/sequential_model/ 
# Sequential Model
model = keras.models.Sequential([
keras.layers.Flatten(input_shape=Xtrain[0].shape),
keras.layers.Dense(100, activation="relu", kernel_initializer="HeNormal"),
keras.layers.Dense(1, activation="elu"),
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9)                 0         
                                                                 
 dense (Dense)               (None, 100)               1000      
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 1,101
Trainable params: 1,101
Non-trainable params: 0
_________________________________________________________________


In [59]:
# Model Compiling 
model.compile(optimizer = 'adam', loss = keras.losses.MeanSquaredError(), metrics = ['mae'])

In [60]:
# Model Fitting
history = model.fit(Xtrain, ytrain, epochs=100, batch_size=30, validation_data=(Xval, yval))

Epoch 1/100
70/70 [==============================] - 1s 7ms/step - loss: 0.4180 - mae: 0.5174 - val_loss: 0.2947 - val_mae: 0.4709
Epoch 2/100
70/70 [==============================] - 0s 4ms/step - loss: 0.2734 - mae: 0.4572 - val_loss: 0.2544 - val_mae: 0.4618
Epoch 3/100
70/70 [==============================] - 0s 3ms/step - loss: 0.2447 - mae: 0.4454 - val_loss: 0.2415 - val_mae: 0.4588
Epoch 4/100
70/70 [==============================] - 0s 4ms/step - loss: 0.2354 - mae: 0.4446 - val_loss: 0.2345 - val_mae: 0.4526
Epoch 5/100
70/70 [==============================] - 0s 3ms/step - loss: 0.2314 - mae: 0.4443 - val_loss: 0.2309 - val_mae: 0.4440
Epoch 6/100
70/70 [==============================] - 0s 5ms/step - loss: 0.2291 - mae: 0.4443 - val_loss: 0.2295 - val_mae: 0.4426
Epoch 7/100
70/70 [==============================] - 0s 5ms/step - loss: 0.2277 - mae: 0.4444 - val_loss: 0.2279 - val_mae: 0.4445
Epoch 8/100
70/70 [==============================] - 0s 7ms/step - loss: 0.2267 - m

In [61]:
# Evaluating The Model
score = model.evaluate(Xtest, ytest, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.2237059324979782
Test accuracy: 0.4483949840068817


In [62]:
# Predicting Output 
y_pred = model.predict(Xtest)
print(y_pred)

33/33 [==============================] - 1s 3ms/step
[[0.3402553 ]
 [0.35990584]
 [0.35272202]
 ...
 [0.30994126]
 [0.34018937]
 [0.34739238]]


# Tune Up The DNN Model 

In [63]:
# Reference: https://keras.io/guides/sequential_model/ 
# Sequential Model
def model_builder(hp):
  model = keras.models.Sequential([
  keras.layers.Flatten(input_shape=Xtrain[0].shape),
  keras.layers.Dense(100, activation="relu", kernel_initializer="HeNormal"),
  keras.layers.Dense(1, activation="relu"),
])
  model.compile(optimizer = 'adam', loss = keras.losses.MeanSquaredError(), metrics = ['mae'])
  history = model.fit(Xtrain, ytrain, epochs=100, batch_size=30, validation_data=(Xval, yval))
  return model

In [64]:
# Instantiate the tuner and perform hypertuning
# Reference: https://keras.io/keras_tuner/ 
tuner = kt.Hyperband(model_builder,
                     objective='val_mae',
                     max_epochs=50,
                     factor=3)

Epoch 1/100
70/70 [==============================] - 1s 5ms/step - loss: 0.2772 - mae: 0.4129 - val_loss: 0.2601 - val_mae: 0.4235
Epoch 2/100
70/70 [==============================] - 0s 3ms/step - loss: 0.2496 - mae: 0.4337 - val_loss: 0.2431 - val_mae: 0.4349
Epoch 3/100
70/70 [==============================] - 0s 4ms/step - loss: 0.2381 - mae: 0.4389 - val_loss: 0.2358 - val_mae: 0.4367
Epoch 4/100
70/70 [==============================] - 0s 5ms/step - loss: 0.2342 - mae: 0.4411 - val_loss: 0.2324 - val_mae: 0.4387
Epoch 5/100
70/70 [==============================] - 0s 3ms/step - loss: 0.2302 - mae: 0.4428 - val_loss: 0.2290 - val_mae: 0.4503
Epoch 6/100
70/70 [==============================] - 0s 3ms/step - loss: 0.2281 - mae: 0.4439 - val_loss: 0.2267 - val_mae: 0.4460
Epoch 7/100
70/70 [==============================] - 0s 5ms/step - loss: 0.2265 - mae: 0.4444 - val_loss: 0.2257 - val_mae: 0.4462
Epoch 8/100
70/70 [==============================] - 0s 5ms/step - loss: 0.2255 - m

In [65]:
# Create a callback to stop training early after reaching a certain value
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_mae', patience=5)

In [66]:
tuner.search(Xtrain, ytrain, epochs=20, validation_split=0.2, callbacks=[stop_early])

In [67]:
# Get the optimal hyperparameters
best_model = tuner.get_best_models()

In [68]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_model)
history = model.fit(Xtrain, ytrain, epochs=20, validation_split=0.2)

val_mae_per_epoch = history.history['val_mae']
best_epoch = val_mae_per_epoch.index(max(val_mae_per_epoch)) +1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/100
70/70 [==============================] - 1s 5ms/step - loss: 0.3151 - mae: 0.3570 - val_loss: 0.3052 - val_mae: 0.3725
Epoch 2/100
70/70 [==============================] - 0s 4ms/step - loss: 0.3001 - mae: 0.3703 - val_loss: 0.3026 - val_mae: 0.3641
Epoch 3/100
70/70 [==============================] - 0s 4ms/step - loss: 0.2985 - mae: 0.3696 - val_loss: 0.3018 - val_mae: 0.3678
Epoch 4/100
70/70 [==============================] - 0s 4ms/step - loss: 0.2980 - mae: 0.3699 - val_loss: 0.3015 - val_mae: 0.3678
Epoch 5/100
70/70 [==============================] - 0s 4ms/step - loss: 0.2980 - mae: 0.3698 - val_loss: 0.3013 - val_mae: 0.3645
Epoch 6/100
70/70 [==============================] - 0s 3ms/step - loss: 0.2977 - mae: 0.3700 - val_loss: 0.3013 - val_mae: 0.3642
Epoch 7/100
70/70 [==============================] - 0s 3ms/step - loss: 0.2976 - mae: 0.3702 - val_loss: 0.3009 - val_mae: 0.3666
Epoch 8/100
70/70 [==============================] - 0s 3ms/step - loss: 0.2974 - m